<a href="https://colab.research.google.com/github/SanjayD23/Alphabetical-Hand-gesture-recognition-using-deep-leaning/blob/main/Alphabetical_Hand_gesture_recognition_using_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as pd
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/MyDrive/Hand_gesture/alphaset"
# This is good